# **Example PSI Model Synthesis Notebook**

Tom Schad 

----

- This notebook does an example synthesis of the Global corona using a Predictive Sciences MHD model. 


----

In [1]:
import pycelp 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
os.environ["XUVTOP"] = '/usr/local/ssw/packages/chianti/dbase/'
import psi

## **Create an instance of the psi.Model class to load and interact with PSI coronal model data**

- Models used here were downloaded using this website from [Predictive Sciences](https://www.predsci.com/hmi/data_access.php)
- We picked the date of the US total solar eclipse (21 Aug 2017) using the med-cor-thermo2 model for this demonstration

In [2]:
## carrington rotation 2194
modelName = 'hmi__med-cor-thermo2-std01__med-hel-poly-std01'
corona = psi.Model('/home/tschad/Dropbox/psiData_21Aug2017_thermo2_12UTC/corona/')

In [3]:
## return the instance name for some basic information on the class
corona

psi Model class
    ---------------------
    Data Directory Names: /home/tschad/Dropbox/psiData_21Aug2017_thermo2_12UTC/corona/
    Number of longitude samples: 181
    Number of latitude samples: 100
    Number of radial samples: 150
    Data shape: (181, 100, 150)
    
    Variables: 
    lons -- Longitudes
    lats -- Latitudes
    rs   -- Radial samples 
    br,bt,bp  -- Spherical components of magnetic field 
    bx,by,bz  -- Cartesian components of magnetic field 
    bmag      -- total magnetic field intensity
    thetaBlocal -- location inclination of magnetic field in solar frame
    

In [4]:
kk = 25

fig,ax = plt.subplots(2,2,figsize = (8,5))
ax = ax.flatten()
ax[0].imshow(corona.bmag[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[1].imshow(corona.thetaBlocal[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[2].imshow(corona.temp[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[3].imshow(corona.ne[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
labels = 'Magnetic Field Intensity','Local B Inclination','Temperature','Electron Density'
for n in range(4): ax[n].set_title(labels[n])

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nRadial Coordinate: ' + str(corona.rs[kk]))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## **Initialize the pyCELP model of the line or lines to be synthesized**

- Here we start with Fe XIII 1074 nm and keep the model fairly "small" at 50 levels.  The errors incurred by using reduced numbers of atomic levels is addressed in [Schad & Dima (2020)](https://rdcu.be/b5J2X) 


In [5]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


In [9]:
# should I parallelize these calculations right away? 
print(181*100.*150. * 484e-6 /80.)  ## seconds on 80 cores

16.42575


In [10]:
wvair = 10747. 

for ii in range(0,181,10): 
    print(ii)
    for jj in range(0,100,10): 
        for kk in range(0,150,10): 
            edens = corona.ne[ii,jj,kk]
            etemp = corona.temp[ii,jj,kk]
            ht = (corona.rs[kk]-1).clip(1.e-8)
            thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
            
            fe13.calc_rho_sym(edens,etemp,ht, thetab)
            upper_lev_rho00 = fe13.get_upper_level_rho00(wvair)
            upper_lev_alignment = fe13.get_upper_level_alignment(wvair)
            total_ion_population = fe13.totn

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180


In [11]:
## LOS integration of polarized emission coefficients 

In [12]:
## INPUTS
obsLon     = 185.     ## observer's longitude
b0         = -5.57    ## observer's heliographic latitude -- b0 angle
Obs_Sun_AU = 1.       ## replace with ephermeris data
fov_rsun   = 6.       ## +/- 3 rsun
arcsamp    = 10.      ## sampling in arcsecond
rsunarc    = 960.     ## radius of sun in arcseconds .. later replace with sun ephemeris

In [16]:

##################
## recall the simulation geometry and plot
## compare to GONG maps of the CR 2189

rObs     = Obs_Sun_AU * (1.495978707e11/6.96340e8)
thetaObs = np.pi/2. - np.deg2rad(b0)
phiObs   = np.deg2rad(obsLon)
xObs,yObs,zObs = rObs*np.sin(thetaObs)*np.cos(phiObs),rObs*np.sin(thetaObs)*np.sin(phiObs),rObs*np.cos(thetaObs)

lonsd = np.rad2deg(corona.lons)
latsd = np.flip(np.rad2deg(corona.lats)-90.)  ## flip as in observers frame angles increase towards north
extrad = (corona.lons[0],corona.lons[-1],corona.lats[-1],corona.lats[0])
extdeg = (lonsd[0],lonsd[-1],latsd[-1],latsd[0])

fig,ax = plt.subplots(nrows = 2,ncols = 1,figsize = (6,8))
ax = ax.flatten()
ax[0].imshow(corona.br[:,:,0].T,extent = extrad)
ax[1].imshow(corona.br[:,:,0].T,extent = extdeg)
ax[2].imshow(corona.z3d[:,:,0].T,extent = extrad)

ax[0].plot(np.zeros(1)+phiObs,np.zeros(1)+thetaObs,'x',markersize = 5,color= 'black')
ax[1].plot(np.zeros(1)+phiObs,np.zeros(1)+thetaObs,'x',markersize = 5,color= 'black')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
from scipy.spatial.transform import Rotation as R

## setup the synthesized field of view

yarc = np.linspace(-fov_rsun/2.*rsunarc,fov_rsun/2.*rsunarc,np.int(np.ceil(fov_rsun*rsunarc/arcsamp)))
zarc = yarc
yya,zza  = np.meshgrid(yarc,zarc,indexing = 'ij')
rra = np.sqrt(yya**2. + zza**2.)
m_behind_sun = 1.*(rra>rsunarc)

################
## define new coordinate system with x towards
## observer, and z towards north pole
## get points in the plane

xxObs = np.zeros_like(yya)
yyObs = rObs * np.tan(np.deg2rad(yya/3600.))
zzObs = rObs * np.tan(np.deg2rad(zza/3600.))

## rotate these points into the model geometry with Euler rotation
r = R.from_euler('yz',[-(thetaObs-np.pi/2.),-(phiObs)])
b = np.stack((xxObs.flatten(),yyObs.flatten(),zzObs.flatten()))
xyz_model = np.matmul(r.as_matrix(), b)
plt.figure()
plt.imshow(xyz_model[1,:].reshape(xxObs.shape).T,origin = 'lower')

## NOW WITH THE XYZ_MODEL POINTS AND THE LOCATION OF THE OBSERVER,
## come up with the parametric equations for the los of sight
## and then get the spherical coordinates, interpolate for rho and temps, etc.
## synthesize and integrate for integrated I,Q,U, (V?)

losvec = np.stack((xObs-xyz_model[0,:],yObs-xyz_model[1,:],zObs - xyz_model[2,:]))
losveclen = np.linalg.norm(losvec,axis=0,ord=2,keepdims = True)
losvec = losvec / losveclen
startpt = np.stack((xyz_model[0,:],xyz_model[1,:],xyz_model[2,:]))


/tmp/ipykernel_106536/491608074.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  yarc = np.linspace(-fov_rsun/2.*rsunarc,fov_rsun/2.*rsunarc,np.int(np.ceil(fov_rsun*rsunarc/arcsamp)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
## generating polarized spectra